In [1]:
# Example 6-2
##(5.2)
# PyTorch Library
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn Library
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Pandas Library
import pandas as pd


In [2]:
##(5.3) load wine data
wine = load_wine() 

In [3]:
##(5.4) print data using DataFrame
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.640000,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.380000,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.680000,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.800000,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.320000,1.04,2.93,735.0
5,14.20,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.750000,1.05,2.85,1450.0
6,14.39,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.250000,1.02,3.58,1290.0
7,14.06,2.15,2.61,17.6,121.0,2.60,2.51,0.31,1.25,5.050000,1.06,3.58,1295.0
8,14.83,1.64,2.17,14.0,97.0,2.80,2.98,0.29,1.98,5.200000,1.08,2.85,1045.0
9,13.86,1.35,2.27,16.0,98.0,2.98,3.15,0.22,1.85,7.220000,1.01,3.55,1045.0


In [4]:
##(5.5) print target 
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [5]:
##(5.6) assign 130 of data and target into variable
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [6]:
##(5.7) divide dataset: 80% of training set and 20% of test set
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)
print(len(train_X))
print(len(test_X))

104
26


In [7]:
##(5.8) convert numpy to torch tensor
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()
print(train_X.shape)
print(train_Y.shape)

torch.Size([104, 13])
torch.Size([104])


In [8]:
##(5.9) merge data and target into one tensor
### TensorDataset: Dataset wrapping tensors that have the same size of first dimension.
train = TensorDataset(train_X, train_Y) 
print(train[0])
### DataLoader: set minibatches which are size of 16 and randomly shuffled
train_loader = DataLoader(train, batch_size=16, shuffle=True)

(tensor([1.3410e+01, 3.8400e+00, 2.1200e+00, 1.8800e+01, 9.0000e+01, 2.4500e+00,
        2.6800e+00, 2.7000e-01, 1.4800e+00, 4.2800e+00, 9.1000e-01, 3.0000e+00,
        1.0350e+03]), tensor(0))


In [9]:
##(5.10) Establish Neural Network 
### class torch.nn.Module: Base class for all NN modules
### Net class inherits nn.Module(parent class)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13,96) 
        self.fc2 = nn.Linear(96,96)
        self.fc3 = nn.Linear(96,96)
        self.fc4 = nn.Linear(96,96)
        self.fc5 = nn.Linear(96,96)
        self.fc6 = nn.Linear(96, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))           ## Hidden layer: ReLU 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)               
        return F.log_softmax(x,dim=0)    ## Output layer: softmax
    
### create instance
model = Net()

In [10]:
##(5.11)
### loss function: cross entropy
criterion = nn.CrossEntropyLoss()

### optimizer: stochastic gradient descent, learning rate 1%
optimizer = optim.SGD(model.parameters(), lr=0.01)

### start learning 
for epoch in range(300):
    total_loss = 0
    ### pop minibatche set out from train_loader
    for train_x, train_y in train_loader:
        ### graph 
        train_x, train_y = Variable(train_x), Variable(train_y)
        ### initialize gradient as PyTorch accumulates gradients
        optimizer.zero_grad()
        ### calc. forward propagation
        output = model(train_x)
        ### calc. loss function
        loss = criterion(output, train_y)
        ### calc. backward propagation
        loss.backward()
        ### update weights
        optimizer.step()
        ### calc. cumulative loss func
        total_loss += loss.data.item()
    if (epoch+1)%50 == 0:
        print(epoch+1, total_loss)

50 1.357663333415985
100 1.0504024773836136
150 1.6780173666775227
200 0.8331075273454189
250 1.7968269251286983
300 1.268087450414896


In [11]:
##(5.12) 
### graph
### torch.autograd.Variable(data): wrapping tensor and logging 
test_x, test_y = Variable(test_X), Variable(test_Y)
### make output 0 or 1
result = torch.max(model(test_x).data, 1)[1]
### accuracy of model
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

accuracy

0.8076923076923077